## Settings

In [1]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [2]:
# ロギング用
import os
from datetime import datetime, timezone, timedelta
from logs.logger import create_logger, get_logger
# モデル作成補助用
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from features.funcs import load_feather
from evals.funcs import get_pred_result, get_acc_and_logloss, print_conf_matrix
from inputs.funcs import load_data
# モデル作成用
from models.keras import ModelKeras
from keras.layers import Dense, Dropout

In [3]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

In [4]:
# loggingの設定を行う
today = datetime.now(timezone(timedelta(hours=9)))
exp_version = today.strftime('%Y%m%d')
os.environ['exp_version'] = exp_version
create_logger(exp_version)

## Keras

### Base

In [5]:
train_x, train_y, test_x = load_data()

In [6]:
train_x = train_x.drop(['Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]
test_x = test_x.drop(['Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
test_x = test_x.dropna()

In [7]:
base_path = '../features/nn/'
cols_to_rplc_train = {'Age': base_path + 'age_train_std_te.feather'}
cols_to_rplc_test = {'Age': base_path + 'age_test_std_te.feather'}
train_x = load_feather(train_x, cols_to_rplc_train)
test_x = load_feather(test_x, cols_to_rplc_test)

In [8]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epoch=10, verbose=0)

[INFO] 2021-02-11 14:44:32,996 >>	=== NN KERAS MODEL ===


In [9]:
pred_train, preds_test = get_pred_result(model, train_x, train_y, test_x)

In [10]:
get_acc_and_logloss(pred_train, train_y, logging=False)

data size: 714
correct predictions: 441
accuracy: 0.6176470588
logloss: 0.7487236281


In [30]:
get_acc_and_logloss(pred_train, train_y, logging=False)
# print_conf_matrix(pred_train, train_y)

data size: 714
correct predictions: 490
accuracy: 0.6862745098
logloss: 0.6398117990
